<a href="https://colab.research.google.com/github/mranas12345/assignment1/blob/main/Generate_QA_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pipeline transformers

In [ ]:
from transformers import pipeline

# Load a Question Generator model
qa_generator = pipeline("text2text-generation", model="t5-base")

text = "The Constitution of Pakistan guarantees the right to a fair trial under Article 10A."

question = qa_generator(f"Generate a question from: {text}")
print(question)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': 'Generate a question from: The Constitution of Pakistan guarantees the right to a fair trial under'}]


In [ ]:
!pip install PyPDF2 PdfReader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.5/292.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.9 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

def extract_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

pdf_text = extract_text("/content/constitution.pdf")
print(pdf_text[:500])  # Print first 500 characters



THE 
CONSTITUTION 
OF THE 
ISLAMIC REPUBLIC 
OF PAKISTAN 
[As modified upto the 31 st May, 2018] 
NATIONAL ASSEMBLY OF PAKISTAN 
 PREFACE 
The National Assembly of Pakistan passed the Constitution on 
10th April, 1973, the President of the Assembly authenticated it on 12th 
April, 1973 and the Assembly published the Constitution of the 
Islamic Republic of Pakistan. Since then, a number of amendments have 
been made therein and it has become necessary and expedient that an 
up-to-date and authen


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 8.4 MB/s eta 0:00:00


In [ ]:
from groq import Groq  # Import the Groq library

# Initialize the Groq client with your API key
client = Groq(api_key="PASTE_API_KEY")  # Replace with your actual API key

def generate_QA(text):
    """Generates question-answer pairs using Groq API."""
    prompt = f"Generate 100 question-answer pairs from the following legal text:\n{text}"

    response = client.chat.completions.create(
        model="llama3-70b-8192",  # Ensure the correct Groq model name
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=5000
    )

    # Extract response content
    return response.choices[0].message.content

# Example input text
pdf_text = "The Constitution of Pakistan guarantees the right to a fair trial under Article 10A."

# Generate QA pairs
qa_pairs = generate_QA(pdf_text)
print(qa_pairs)


Here are 100 question-answer pairs based on the given legal text:

**Q1:** What is guaranteed by the Constitution of Pakistan?
**A1:** The right to a fair trial.

**Q2:** Which article of the Constitution of Pakistan guarantees the right to a fair trial?
**A2:** Article 10A.

**Q3:** What is the significance of Article 10A of the Constitution of Pakistan?
**A3:** It guarantees the right to a fair trial.

**Q4:** Is the right to a fair trial protected in Pakistan?
**A4:** Yes, it is guaranteed by the Constitution of Pakistan.

**Q5:** What is the source of the right to a fair trial in Pakistan?
**A5:** The Constitution of Pakistan.

**Q6:** Which provision of the Constitution of Pakistan deals with the right to a fair trial?
**A6:** Article 10A.

**Q7:** Is the right to a fair trial a fundamental right in Pakistan?
**A7:** Yes, it is guaranteed by the Constitution of Pakistan.

**Q8:** What does Article 10A of the Constitution of Pakistan ensure?
**A8:** A fair trial.

**Q9:** Is the ri

In [ ]:
import json
from groq import Groq
from PyPDF2 import PdfReader
from transformers import pipeline

# Initialize the Groq client with your API key
client = Groq(api_key="Replace with your actual API key")  # Replace with your actual API key

# Function to extract text from a PDF
def extract_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

# Function to split text into chunks for better processing
def split_text(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Function to generate multiple QA pairs using Groq API
def generate_QA(text, num_questions=5):
    """Generates question-answer pairs using Groq API."""
    prompt = f"Generate {num_questions} question-answer pairs from the following legal text:\n{text}"

    response = client.chat.completions.create(
        model="llama3-70b-8192",  # Ensure the correct Groq model name
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=700
    )

    return response.choices[0].message.content

# Load a Question Generator model for extra QA generation
qa_generator = pipeline("text2text-generation", model="t5-base")

# Function to generate extra questions from text
def generate_extra_questions(text, num_questions=5):
    questions = []
    for _ in range(num_questions):
        q = qa_generator(f"Generate a question from: {text}", max_length=100, truncation=True)
        questions.append(q[0]['generated_text'])
    return questions

# Extract text from PDF
pdf_text = extract_text("/content/constitution.pdf")

# Split text into chunks
text_chunks = split_text(pdf_text, chunk_size=500)

# Generate QA pairs for each chunk
qa_dataset = []
for i, chunk in enumerate(text_chunks):
    print(f"Processing chunk {i+1}/{len(text_chunks)}...")

    try:
        # Generate QA from Groq API
        groq_qa = generate_QA(chunk, num_questions=10)  # Increase number of questions per chunk

        # Generate additional questions using T5
        extra_questions = generate_extra_questions(chunk, num_questions=5)

        qa_dataset.append({"chunk": chunk, "groq_qa": groq_qa, "extra_questions": extra_questions})

    except Exception as e:
        print(f"Error processing chunk {i+1}: {e}")

# Save dataset as JSON
with open("qa_dataset.json", "w") as f:
    json.dump(qa_dataset, f, indent=4)

print("✅ QA Dataset generated successfully!")


Device set to use cuda:0


Processing chunk 1/139...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Processing chunk 2/139...
Processing chunk 3/139...
Processing chunk 4/139...
Processing chunk 5/139...
Processing chunk 6/139...
Processing chunk 7/139...
Processing chunk 8/139...
Processing chunk 9/139...
Processing chunk 10/139...
Processing chunk 11/139...
Processing chunk 12/139...
Processing chunk 13/139...
Processing chunk 14/139...
Processing chunk 15/139...
Processing chunk 16/139...
Processing chunk 17/139...
Processing chunk 18/139...
Processing chunk 19/139...
Processing chunk 20/139...
Processing chunk 21/139...
Processing chunk 22/139...
Processing chunk 23/139...
Processing chunk 24/139...
Processing chunk 25/139...
Processing chunk 26/139...
Processing chunk 27/139...
Processing chunk 28/139...
Processing chunk 29/139...
Processing chunk 30/139...
Processing chunk 31/139...
Processing chunk 32/139...
Processing chunk 33/139...
Processing chunk 34/139...
Processing chunk 35/139...
Processing chunk 36/139...
Processing chunk 37/139...
Processing chunk 38/139...
Processin

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Function to save dataset as a PDF
def save_as_pdf(qa_data, filename="qa_dataset.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    y_position = height - 40  # Start position for text

    c.setFont("Helvetica", 10)
    c.drawString(30, y_position, "Generated QA Dataset")
    y_position -= 20

    for i, entry in enumerate(qa_data):
        if y_position < 40:  # Move to next page if space runs out
            c.showPage()
            c.setFont("Helvetica", 10)
            y_position = height - 40

        c.drawString(30, y_position, f"Chunk {i+1}:")
        y_position -= 15
        c.drawString(40, y_position, f"Text: {entry['chunk']}")
        y_position -= 15
        c.drawString(40, y_position, f"Generated QA Pairs: {entry['groq_qa']}")
        y_position -= 15
        c.drawString(40, y_position, f"Extra Questions: {', '.join(entry['extra_questions'])}")
        y_position -= 20

    c.save()
    print(f"✅ QA Dataset saved successfully as {filename}")

# Save dataset as PDF
save_as_pdf(qa_dataset)
